# Step 3. Combine NDVI, OSM, and GSV data

In [ ]:
import os
import datetime

import numpy as np
import scipy
import fiona
import statistics
import math

import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString, shape, mapping, Point, Polygon, MultiPolygon
from shapely.ops import cascaded_union, transform
import pyproj

import matplotlib.pyplot as plt
from matplotlib import colors, cm, style
import matplotlib.patches as mpatches
# from descartes import PolygonPatch

import osmnx as ox
import networkx as nx

import rasterio
from rasterio import MemoryFile
from rasterio.plot import show
from rasterio.mask import mask
from rasterio.features import shapes
import json

import contextily as cx
import folium
from folium.features import DivIcon

import random

In [ ]:
from getpass import getpass

import requests
from requests import Request, Session

import hashlib
import hmac
import base64
import urllib.parse as urlparse

from datetime import date
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
obtain_imagery = False

In [ ]:
api_key = getpass("API key: ")
if obtain_imagery:
    secret = getpass("URL signing secret: ")

## Define city and other settings

#### Place |Country    |CRS
Rotterdam/Utrecht    (The Netherlands)    EPSG:28992

Barcelona     (Spain)              EPSG:25830

Goteborg        (Sweden)             EPSG:3006

In [ ]:
place_name = 'Barcelona'
local_crs = 'EPSG:25830'

In [ ]:
n_presample = 1000

In [ ]:
osm_crs = 'EPSG:3857'
gsv_crs = 'EPSG:4326'

In [ ]:
export_folder = 'data'
date_sub_folder = '{}_15Mar2023'.format(place_name.split(',')[0].replace(' ', ''))

In [ ]:
suitable_season = {
    'start': '05',
    'end': '09'
}
suitable_years = {
    'start': '2018',
    'end': '2022'
}

In [ ]:
buffer = 500

In [ ]:
# set random seed for generating random numbers
# and for sampling rows from geodataframes
random_state = 42
random.seed(random_state)

## Read NDVI data

In [ ]:
geotiff_file = os.path.join(export_folder, 'NDVI', 'NDVI cloudless {} 10mres.tif'.format(place_name))
geotiff = rasterio.open(geotiff_file)
geotiff_data = geotiff.read()

## Read OSM data and define OSM categories

Greenspace sizes:
- pocket park: 200 sq.m. <= area < 500 sq.m.
- regular size greenspace: 500 sq.m. <= area

In [ ]:
greenspaces = gpd.read_file(os.path.join(export_folder, 'OSM', date_sub_folder, 'greenspaces.geojson'))
greenspaces.crs = local_crs

In [ ]:
#union of adjacent and overlapping polygon greenspaces
greenspaces = gpd.GeoDataFrame(greenspaces.unary_union).rename(columns={0:'geometry'}).set_geometry('geometry').set_crs(local_crs)

In [ ]:
area_thresh_0 = 200
area_thresh_1 = 5000

In [ ]:
print('pocket:  ', len(greenspaces[greenspaces.area<area_thresh_1]))
print('regular: ', len(greenspaces[greenspaces.area>=area_thresh_1]))
print('TOTAL:   ', len(greenspaces))

In [ ]:
pocket_greenspaces = greenspaces[
    (greenspaces.area>=area_thresh_0) &
    (greenspaces.area<area_thresh_1)].copy()
pocket_greenspaces['category'] = 'pocket_greenspace'

In [ ]:
regular_greenspaces = greenspaces[greenspaces.area>=area_thresh_1].copy()
regular_greenspaces['category'] = 'regular_greenspace'

In [ ]:
squares = gpd.read_file(os.path.join(export_folder, 'OSM', date_sub_folder, 'squares.geojson'))
squares['category'] = 'square'
squares.crs = local_crs

In [ ]:
playspaces = gpd.read_file(os.path.join(export_folder, 'OSM', date_sub_folder, 'playspaces.geojson'))
playspaces['category'] = 'playspace'
playspaces.crs = local_crs

In [ ]:
streets = gpd.read_file(os.path.join(export_folder, 'OSM', date_sub_folder, 'streets.geojson'))
streets['category'] = 'street'
streets.crs = local_crs

In [ ]:
# take only longer streets, in line with minimum area of other places
# length of 100 plus buffer of 10m (later on for NDVI's) makes ~ 100*20 = 2000 sq.m. = 0.2 hectare
length_thresh = 100
streets = streets[streets.geometry.length>=length_thresh].copy()

In [ ]:
parkings = gpd.read_file(os.path.join(export_folder, 'OSM', date_sub_folder, 'parkings.geojson'))
parkings['category'] = 'parking'
parkings.crs = local_crs

In [ ]:
place = gpd.read_file(os.path.join(export_folder, 'OSM', date_sub_folder, 'place.geojson'))
place.crs = local_crs

#### secondary categories

In [ ]:
def add_secondary_categories(gdf, other_gdfs, threshold=10):
    # if within 10 meters from another category, store also that category as secondary category
    
    cols=[]
    
    for i in range(len(other_gdfs)):
        other = other_gdfs[i][['category', 'geometry']]
        col = 'category_{}'.format(i+2)
        cols.append(col)
        gdf = gdf.sjoin(other.set_geometry(other.geometry.buffer(threshold)).rename(columns={'category': col}), how='left').drop(columns={'index_right'}).drop_duplicates(subset='geometry')
        
    gdf['secondary_categories'] = gdf.apply(lambda row: [x for x in row[cols] if x == x], axis=1)
    gdf.drop(cols, axis=1, inplace=True)
    
    return gdf

In [ ]:
pocket_greenspaces = add_secondary_categories(pocket_greenspaces, [regular_greenspaces, squares, playspaces, streets, parkings])
regular_greenspaces = add_secondary_categories(regular_greenspaces, [pocket_greenspaces, squares, playspaces, streets, parkings])
squares = add_secondary_categories(squares, [pocket_greenspaces, regular_greenspaces, playspaces, streets, parkings])
playspaces = add_secondary_categories(playspaces, [pocket_greenspaces, regular_greenspaces, squares, streets, parkings])
streets = add_secondary_categories(streets, [pocket_greenspaces, regular_greenspaces, squares, playspaces, parkings])
parkings = add_secondary_categories(parkings, [pocket_greenspaces, regular_greenspaces, squares, playspaces, streets])

#### plot

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(12,12))

show((geotiff, 1), cmap='RdYlGn', ax=axs, zorder=1)

pocket_greenspaces.to_crs(geotiff.crs).plot(ax=axs, facecolor='none', edgecolor='green')
regular_greenspaces.to_crs(geotiff.crs).plot(ax=axs, facecolor='none', edgecolor='green')

squares.to_crs(geotiff.crs).plot(ax=axs, facecolor='none', edgecolor='pink')
playspaces.to_crs(geotiff.crs).plot(ax=axs, facecolor='none', edgecolor='orange') 
streets.to_crs(geotiff.crs).plot(ax=axs, facecolor='none', edgecolor='purple', linewidth=0.2, zorder=2)

parkings.to_crs(geotiff.crs).plot(ax=axs, facecolor='none', edgecolor='black')

place.to_crs(geotiff.crs).plot(ax=axs, facecolor='none', edgecolor='#dd1c77', linestyle='-', linewidth=5, zorder=4)
# cx.add_basemap(ax=axs, crs=geotiff.crs, source=cx.providers.OpenStreetMap.Mapnik, alpha=0.5, zorder=0)

bbox = place.to_crs(geotiff.crs).total_bounds
margin = 0
xlim = ([bbox[0]-margin, bbox[2]+margin])
ylim = ([bbox[1]-margin, bbox[3]+margin])
axs.set_xlim(xlim)
axs.set_ylim(ylim)
# plt.axis('off')

plt.show()

## Read population data and determine OSM places in proximity

Global Human Settlement Layer (GHSL) Settlement Model for 2022:

https://ghsl.jrc.ec.europa.eu/ghs_smod2022.php

National datasets downloaded via

- for The Netherlands: 2022
    - https://www.cbs.nl/nl-nl/dossier/nederland-regionaal/geografische-data/wijk-en-buurtkaart-2022
- for Barcelona: 2022
    - geometries 2022 from https://www.ine.es/ss/Satellite?c=Page&p=1259952026632&pagename=ProductosYServicios%2FPYSLayout&cid=1259952026632&L=1
    - population 2022 from https://ajuntament.barcelona.cat/estadistica/castella/Estadistiques_per_temes/Poblacio_i_demografia/Poblacio/Padro_municipal_habitants/a2022/sexe/sexe05.htm
- for Sweden: 2021
    - geometries 2018 (= most recent, still valid for 2021) from https://scb.se/vara-tjanster/oppna-data/oppna-geodata/deso--demografiska-statistikomraden/
    - population data 2021 from https://www.statistikdatabasen.scb.se/pxweb/en/ssd/START__BE__BE0101__BE0101Y/FolkmDesoAldKonN/
    

In [ ]:
if 'Rotterdam' in place_name or 'Goteborg' in place_name:
    filename = 'GHS_SMOD_E2020_GLOBE_R2022A_54009_1000_V1_0_R3_C19'
elif 'Barcelona' in place_name:
    filename = 'GHS_SMOD_E2020_GLOBE_R2022A_54009_1000_V1_0_R5_C19'

# read geotiff data
filepath = os.path.join(export_folder, 'population', filename, filename+'.tif')
settlementmodel = rasterio.open(filepath)
settlementmodel_data = settlementmodel.read(1, masked=True)

# Use a generator instead of a list
shape_gen = ((shape(s), v) for s, v in shapes(settlementmodel_data, transform=settlementmodel.transform))

# or build a dict from unpacked shapes
settlementmodel_gdf = gpd.GeoDataFrame(dict(zip(["geometry", "class"], zip(*shape_gen))), crs=settlementmodel.crs)

# convert to crs and filter out urban centres
urbancentre = 30
urbancentres = settlementmodel_gdf[settlementmodel_gdf['class']==urbancentre]
urbancentres = urbancentres.to_crs(local_crs)
urbancentres = gpd.clip(urbancentres, place)

In [ ]:
if 'Rotterdam' in place_name:
    # load shapefile with geometries and population data
    population = gpd.read_file(os.path.join(export_folder, 'population', 'Rotterdam', 'buurt_2022_v1.shp'))
    population = population[population['GM_NAAM']=='Rotterdam']
    # exclude harbour and water areas
    population = population[(population['OPP_LAND'] > 0)]
    population = population.sjoin(place, how='inner')
    population = population.replace({'AANT_INW': -99999999}, value=0)
    population['population'] = population['AANT_INW']
    
elif 'Barcelona' in place_name:
    # load shapefile with census geometries
    geometries = gpd.read_file(os.path.join(export_folder, 'population', 'Barcelona', 'SECC_CE_20220101.shp'))
    geometries = geometries[geometries['NMUN']=='Barcelona']
    geometries['CDIS'] = geometries['CDIS'].astype(int)
    geometries['CSEC'] = geometries['CSEC'].astype(int)
    # load table with population data
    table = pd.read_excel(os.path.join(export_folder, 'population', 'Barcelona', 'population_barcelona_2022_preprocessed.xlsx'))
    table = table[table['CDIS_CSEC']!='BARCELONA']
    table['CDIS_CSEC'] = table['CDIS_CSEC'].str.replace('\xa0', ' ').str.replace('     ', '-').str.replace('  ', '')
    table['TOTAL'] = table['TOTAL'].str.replace('\xa0', '').str.replace('.', '').astype(int)
    table['CDIS'] = table['CDIS_CSEC'].str.split('-', expand=True)[0].astype(int)
    table['CSEC'] = table['CDIS_CSEC'].str.split('-', expand=True)[1].astype(int)
    # merge them and calculate density in sq.km.
    population = geometries.merge(table, how='left', on=['CDIS', 'CSEC'])
    population['population'] = population['TOTAL']
    
elif 'Goteborg' in place_name:
    # load shapefile with census geometries
    geometries = gpd.read_file(os.path.join(export_folder, 'population', 'Goteborg', 'DeSO_2018_v2.gpkg'))
    # load table with population data
    table = pd.read_excel(os.path.join(export_folder, 'population', 'Goteborg', 'deso_poulation_2021.xlsx'))
    table['total_2021'] = table['total_2021'].str.replace(' ', '')
    table['total_2021'] = table['total_2021'].fillna(0)
    # merge them 
    population = geometries.merge(table, on='deso', how='left')
    population['population'] = population['total_2021'].astype(float)

# calculate density in sq.km.
population['population_density'] = population.population / (population.geometry.area / 1000000)

In [ ]:
# ref see https://ghsl.jrc.ec.europa.eu/data.php#GHSLBasics
pop_dens_threshold = 300

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(12,12))

population[population.population_density>=pop_dens_threshold].plot(ax=axs, column='population_density', cmap='plasma', figsize=(10,10), scheme='percentiles', legend=True, alpha=0.5)
population[population.population_density<pop_dens_threshold].plot(ax=axs, color='grey', alpha=0.5)
urbancentres.plot(ax=axs, facecolor='None', edgecolor='black', linewidth=5, alpha=0.5)
cx.add_basemap(ax=axs, crs=local_crs, source=cx.providers.OpenStreetMap.Mapnik, alpha=0.5, zorder=0)
place.plot(ax=axs, edgecolor='black', color='None')


plt.show()

In [ ]:
population = population[population['population_density']>pop_dens_threshold]

In [ ]:
def check_zone_of_interest(gdf):
    # check if intersects with place
    gdf = gdf.sjoin(place.set_geometry(place.geometry.buffer(buffer))[['geometry']], how='inner', predicate='intersects')
    gdf = gdf.drop(columns=['index_right'])
    gdf = gdf.drop_duplicates(subset='geometry', keep='first')
    # check if intersects with urban centres
    gdf = gdf.sjoin(urbancentres.set_geometry(urbancentres.geometry.buffer(buffer))[['geometry']], how='inner', predicate='intersects')
    gdf = gdf.drop(columns=['index_right'])
    gdf = gdf.drop_duplicates(subset='geometry', keep='first')
    # check if intersects with populated zone
    gdf = gdf.sjoin(population.set_geometry(population.geometry.buffer(buffer))[['geometry']], how='inner', predicate='intersects')
    gdf = gdf.drop(columns=['index_right'])
    gdf = gdf.drop_duplicates(subset='geometry', keep='first')
    
    return gdf

In [ ]:
pocket_greenspaces = check_zone_of_interest(pocket_greenspaces)
regular_greenspaces = check_zone_of_interest(regular_greenspaces)
squares = check_zone_of_interest(squares)
playspaces = check_zone_of_interest(playspaces)
streets = check_zone_of_interest(streets)
parkings = check_zone_of_interest(parkings)

#### Presample if too many places within a category

In [ ]:
len_all = (
    len(pocket_greenspaces) + len(regular_greenspaces) + 
    len(squares) + len(playspaces) + len(streets) + len(parkings))

In [ ]:
print(
    '{} pocket-size greenspaces ({}%)\n'.format(str(len(pocket_greenspaces)), round(100*len(pocket_greenspaces)/len_all,2)) +
    '{} regular-size greenspaces ({}%)\n'.format(str(len(regular_greenspaces)), round(100*len(regular_greenspaces)/len_all,2)) +
    '{} squares ({}%)\n'.format(str(len(squares)), round(100*len(squares)/len_all,2)) +
    '{} playspaces ({}%)\n'.format(str(len(playspaces)), round(100*len(playspaces)/len_all,2)) +
    '{} streets ({}%)\n'.format(str(len(streets)), round(100*len(streets)/len_all,2)) +
    '{} parkings ({}%)'.format(str(len(parkings)), round(100*len(parkings)/len_all,2))
)

In [ ]:
if len(pocket_greenspaces)>n_presample:
    print('sampling {}/{} random pocket-size greenspaces'.format(n_presample, len(pocket_greenspaces)))
    pocket_greenspaces = pocket_greenspaces.sample(n=n_presample, random_state=random_state)
    
if len(regular_greenspaces)>n_presample:
    print('sampling {}/{} random regular-size greenspaces'.format(n_presample, len(regular_greenspaces)))
    regular_greenspaces = regular_greenspaces.sample(n=n_presample, random_state=random_state)
    
if len(squares)>n_presample:
    print('sampling {}/{} random squares'.format(n_presample, len(squares)))
    squares = squares.sample(n=n_presample, random_state=random_state)
    
if len(playspaces)>n_presample:
    print('sampling {}/{} random playspaces'.format(n_presample, len(playspaces)))
    playspaces = playspaces.sample(n=n_presample, random_state=random_state)
    
if len(streets)>n_presample:
    print('sampling {}/{} random streets'.format(n_presample, len(streets)))
    streets = streets.sample(n=n_presample, random_state=random_state)
    
if len(parkings)>n_presample:
    print('sampling {}/{} random parkings'.format(n_presample, len(parkings)))
    parkings = parkings.sample(n=n_presample, random_state=random_state)

## Combine OSM with GSV

In [ ]:
# for GSV and NDVI enrichments, give streets a buffer zone
# 10 meter buffer around streets to go from lines to polygons
street_buffer = 10
streets = streets.set_geometry(streets.geometry.buffer(street_buffer))

### Get metadata and check if suitable imagery exists here

In [ ]:
meta_base = 'https://maps.googleapis.com/maps/api/streetview/metadata?'
pic_base = 'https://maps.googleapis.com/maps/api/streetview?'

In [ ]:
def points_in_polygon(polygon, n):
    # generate n random points withn the polygon
    points = []
    min_x, min_y, max_x, max_y = polygon.bounds
    i = 0
    while i < n:
        point = Point(random.uniform(min_x, max_x), random.uniform(min_y, max_y))
        if polygon.contains(point):
            points.append(point)
            i += 1
    return points

In [ ]:
def obtain_metadata(location, radius):    
    meta_params = {
        'key': api_key,
        'location': location,
        'radius': radius}
    
    # obtain the metadata of the request (this is free)
    meta_response = requests.get(meta_base, params=meta_params)
    return meta_response.json()

In [ ]:
def get_gsv_availability(row, radius, n):
    
    status = False
    year = None
    month = None
    suitability = False
    pano_id = False
    lat = False
    lng = False
    
    points = points_in_polygon(row.geometry, n)
    
    for point in points:
        location = '{},{}'.format(point.y, point.x)
    
        meta = obtain_metadata(location, radius)

        # check status
        if meta['status'] == 'OK':
            status = True

            # find date and year
            if 'date' in meta:
                date = meta['date'].split('-')
                year = date[0]
                if len(date) > 1:
                    month = date[1]

                    # check suitability based on date and year         
                    if month>=suitable_season['start'] and month<=suitable_season['end'] and year>=suitable_years['start'] and year<=suitable_years['end']:
                        suitability = True
                        pano_id = meta['pano_id']
                        lat = meta['location']['lat']
                        lng = meta['location']['lng']
                        
        if suitability:
            break
    
    return suitability, status, year, month, pano_id, lat, lng

In [ ]:
gsv_cols = ['gsv_suitability', 'gsv_status', 'gsv_year', 'gsv_month', 'gsv_pano_id', 'gsv_lat', 'gsv_lng']

In [ ]:
# GSV radius 50m default, we go with less, e.g., 15, 29 or 43
gsv_radius = 15
# look for GSV imagery around n sample points within each place
n = 10

In [ ]:
%%time
pocket_greenspaces[gsv_cols] = pocket_greenspaces.to_crs(gsv_crs).apply(lambda row: get_gsv_availability(row, gsv_radius, n), axis=1, result_type="expand")

In [ ]:
%%time
regular_greenspaces[gsv_cols] = regular_greenspaces.to_crs(gsv_crs).apply(lambda row: get_gsv_availability(row, gsv_radius, n), axis=1, result_type="expand")

In [ ]:
%%time
squares[gsv_cols] = squares.to_crs(gsv_crs).apply(lambda row: get_gsv_availability(row, gsv_radius, n), axis=1, result_type="expand")

In [ ]:
%%time
playspaces[gsv_cols] = playspaces.to_crs(gsv_crs).apply(lambda row: get_gsv_availability(row, gsv_radius, n), axis=1, result_type="expand")

In [ ]:
%%time
streets[gsv_cols] = streets.to_crs(gsv_crs).apply(lambda row: get_gsv_availability(row, gsv_radius, n), axis=1, result_type="expand")

In [ ]:
%%time
parkings[gsv_cols] = parkings.to_crs(gsv_crs).apply(lambda row: get_gsv_availability(row, gsv_radius, n), axis=1, result_type="expand")

In [ ]:
print(
    str(len(pocket_greenspaces[pocket_greenspaces.gsv_suitability])) + ' pocket-size greenspaces with GSV imagery\n' +
    str(len(regular_greenspaces[regular_greenspaces.gsv_suitability])) + ' regular-size greenspaces with GSV imagery\n' +
    str(len(squares[squares.gsv_suitability])) + ' squares with GSV imagery\n' +
    str(len(playspaces[playspaces.gsv_suitability])) + ' playspaces with GSV imagery\n' +
    str(len(streets[streets.gsv_suitability])) + ' streets with GSV imagery\n' +
    str(len(parkings[parkings.gsv_suitability])) + ' parkings with GSV imagery'
)

## Combine OSM with NDVI

In [ ]:
def get_ndvi_values(gdf, geotiff):
    gdf_mask = gdf.to_crs(geotiff.crs).copy()    
    
    nodata = 255

    for i in range(len(gdf_mask)):  
        
        # for each row in the gdf
        coords = [json.loads(gdf_mask.to_json())['features'][i]['geometry']]
        index = int([json.loads(gdf_mask.to_json())['features'][i]['id']][0])
        
        data, out_transform = mask(dataset=geotiff, shapes=coords, filled=True, crop=True, nodata=nodata)
        
        # exclude all nodata values and values below 0 (water)
        data = data[data!=nodata]
        data = data[data>=0]
        
        if len(data)==0:
            gdf_mask.loc[index, 'ndvi_mean'] = None
            gdf_mask.loc[index, 'ndvi_median'] = None
            gdf_mask.loc[index, 'ndvi_min'] = None
            gdf_mask.loc[index, 'ndvi_max'] = None
            gdf_mask.loc[index, 'ndvi_stdev'] = None
            gdf_mask.loc[index, 'ndvi_var'] = None
        else:
            gdf_mask.loc[index, 'ndvi_mean'] = round(np.mean(data), 3)
            gdf_mask.loc[index, 'ndvi_median'] = round(np.median(data), 3)
            gdf_mask.loc[index, 'ndvi_min'] = round(np.min(data), 3)
            gdf_mask.loc[index, 'ndvi_max'] = round(np.max(data), 3)
            gdf_mask.loc[index, 'ndvi_stdev'] = round(np.std(data), 3)
            gdf_mask.loc[index, 'ndvi_var'] = round(np.var(data), 3)
    
    gdf_mask = gdf_mask.to_crs(gdf.crs).copy()  
    return gdf_mask

In [ ]:
%%time
pocket_greenspaces = get_ndvi_values(pocket_greenspaces, geotiff)

In [ ]:
%%time
regular_greenspaces = get_ndvi_values(regular_greenspaces, geotiff)

In [ ]:
%%time
squares = get_ndvi_values(squares, geotiff)

In [ ]:
%%time
playspaces = get_ndvi_values(playspaces, geotiff)

In [ ]:
%%time
streets = get_ndvi_values(streets, geotiff)

In [ ]:
%%time
parkings = get_ndvi_values(parkings, geotiff)

In [ ]:
print(
    str(len(pocket_greenspaces[(pocket_greenspaces.gsv_suitability) & (pocket_greenspaces.ndvi_mean.notna())])) + ' pocket-size greenspaces with GSV imagery and NDVI value\n' +
    str(len(regular_greenspaces[(regular_greenspaces.gsv_suitability) & (regular_greenspaces.ndvi_mean.notna())])) + ' regular-size greenspaces with GSV imagery and NDVI value\n' +
    str(len(squares[(squares.gsv_suitability) & (squares.ndvi_mean.notna())])) + ' squares with GSV imagery and NDVI value\n' +
    str(len(playspaces[(playspaces.gsv_suitability) & (playspaces.ndvi_mean.notna())])) + ' playspaces with GSV imagery and NDVI value\n' +
    str(len(streets[(streets.gsv_suitability) & (streets.ndvi_mean.notna())])) + ' streets with GSV imagery and NDVI value\n' +
    str(len(parkings[(parkings.gsv_suitability) & (parkings.ndvi_mean.notna())])) + ' parkings with GSV imagery and NDVI value'
)

## Visualize enriched OSM

In [ ]:
def add_gdf_to_folium(gdf, color, space_type):
    for _, r in gdf.iterrows():
        # Without simplifying the representation of each borough,
        # the map might not be displayed
        sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.000001)
        geo_j = sim_geo.to_json()
        geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {'fillColor': color})
        folium.Popup("""
            OSM {}<br>
            NDVI {}<br>
            GSV {}/{}
            """.format(
                space_type,
                r['ndvi_mean'],
                r['gsv_month'],
                r['gsv_year']
            )
        ).add_to(geo_j)
        geo_j.add_to(m)

In [ ]:
# Create a folium map object.
center = [place.to_crs(geotiff.crs).geometry.centroid[0].y, place.to_crs(geotiff.crs).geometry.centroid[0].x]
m = folium.Map(location=center, zoom_start=14)

add_gdf_to_folium(pocket_greenspaces.to_crs(geotiff.crs)[pocket_greenspaces['gsv_suitability']], 'green', 'pocket-size greenspace')
add_gdf_to_folium(regular_greenspaces.to_crs(geotiff.crs)[regular_greenspaces['gsv_suitability']], 'green', 'regular-size greenspace')

add_gdf_to_folium(squares.to_crs(geotiff.crs)[squares['gsv_suitability']], 'pink', 'square')
add_gdf_to_folium(playspaces.to_crs(geotiff.crs)[playspaces['gsv_suitability']], 'orange', 'playspace')
add_gdf_to_folium(streets.to_crs(geotiff.crs)[streets['gsv_suitability']], 'purple', 'street')

add_gdf_to_folium(parkings.to_crs(geotiff.crs)[parkings['gsv_suitability']], 'grey', 'parking')

# Display the map.
display(m)

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=5, figsize=(20,4), sharex=True)

axs[0].hist(pocket_greenspaces['ndvi_mean'], color='green', alpha=0.3)
axs[0].hist(regular_greenspaces['ndvi_mean'], color='green', alpha=0.3)
axs[0].set_title('OSM greenspaces (all sizes)')
axs[0].set_xlabel('mean NDVI')

axs[1].hist(squares['ndvi_mean'], color='red', alpha=0.3, zorder=2)
axs[1].set_title('OSM squares')
axs[1].set_xlabel('mean NDVI')

axs[2].hist(playspaces['ndvi_mean'], color='orange', alpha=0.3, zorder=3)
axs[2].set_title('OSM playgrounds (orange)')
axs[2].set_xlabel('mean NDVI')

axs[3].hist(streets['ndvi_mean'], color='purple', alpha=0.3, zorder=1)
axs[3].set_title('OSM streets (purple)')
axs[3].set_xlabel('mean NDVI')


axs[4].hist(parkings['ndvi_mean'], color='black', alpha=0.3)
axs[4].set_title('OSM parkings')
axs[4].set_xlabel('mean NDVI')

plt.show()

## Export enriched OSM data: with NDVI average value and GSV availability columns

In [ ]:
export_sub_folder = os.path.join(export_folder, 'OSM', date_sub_folder, 'enriched')

In [ ]:
if not os.path.exists(export_sub_folder):
    os.mkdir(export_sub_folder)

In [ ]:
output_file = os.path.join(export_sub_folder, 'pocket_greenspaces_enriched.geojson')
pocket_greenspaces_export = pocket_greenspaces.apply(lambda c: c.astype(str) if c.name != "geometry" else c, axis=0)
pocket_greenspaces_export.to_file(output_file, driver='GeoJSON')

In [ ]:
output_file = os.path.join(export_sub_folder, 'regular_greenspaces_enriched.geojson')
regular_greenspaces_export = regular_greenspaces.apply(lambda c: c.astype(str) if c.name != "geometry" else c, axis=0)
regular_greenspaces_export.to_file(output_file, driver='GeoJSON')

In [ ]:
output_file = os.path.join(export_sub_folder, 'squares_enriched.geojson')
squares_export = squares.apply(lambda c: c.astype(str) if c.name != "geometry" else c, axis=0)
squares_export.to_file(output_file, driver='GeoJSON')

In [ ]:
output_file = os.path.join(export_sub_folder, 'playspaces_enriched.geojson')
playspaces_export = playspaces.apply(lambda c: c.astype(str) if c.name != "geometry" else c, axis=0)
playspaces_export.to_file(output_file, driver='GeoJSON')

In [ ]:
output_file = os.path.join(export_sub_folder, 'streets_enriched.geojson')
streets_export = streets.apply(lambda c: c.astype(str) if c.name != "geometry" else c, axis=0)
streets_export.to_file(output_file, driver='GeoJSON')

In [ ]:
output_file = os.path.join(export_sub_folder, 'parkings_enriched.geojson')
parkings_export = parkings.apply(lambda c: c.astype(str) if c.name != "geometry" else c, axis=0)
parkings_export.to_file(output_file, driver='GeoJSON')